In [ ]:
import os
import json
import time
import logging
import azureml
from azureml.core import Workspace, Run, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

## Compute environment

In [ ]:
cluster = 'sauron'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

## Automated Machine learning

In [ ]:
automl_settings = {
    "name": "AUTOMLMNIST_{0}".format(time.time()),
    "iteration_timeout_minutes": 5,
    "iterations": 20,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 10,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             path='.',
                             compute_target = compute,
                             data_script='./get_data.py',
                             **automl_settings)`

In [ ]:
experiment=Experiment(ws, 'mnist_lab')
remote_run = experiment.submit(automl_config)

In [ ]:
#remote_run = [r for r in exp.get_runs() if r.id == 'AutoML_b41af9f3-998c-4f3c-8493-9a076f702631'][0]
RunDetails(remote_run).show()